In [1]:
import util
import numpy as np
import jpeglib
import matplotlib.pyplot as plt
import cv2

In [38]:
def reapply_jpeg(image,shifts,qt,shiftFunc):
    compressed_image=[]
    for dx,dy in shifts:
        shifted_image=shiftFunc(image,dx,dy)
        # print(np.shape(image))
        # print(np.shape(shifted_image))
        jpeglib.version.set('6b')
        im = jpeglib.from_spatial(shifted_image)
        name="temp.jpeg"
        im.jpeg_color_space = jpeglib.JCS_GRAYSCALE
        im.write_spatial(name,qt) #compress with quality N=100 in valid range [0, 100]
        im_quanti = jpeglib.read_spatial(name)
        compressed_image.append(im_quanti.spatial)
    average=np.mean(compressed_image,axis=0).astype(np.uint8)
    jpeglib.version.set('6b')
    img = jpeglib.from_spatial(average)
    name="reapply"+str(shiftFunc)+str(qt)+".jpeg"
    img.jpeg_color_space = jpeglib.JCS_GRAYSCALE
    img.write_spatial(name,100)
    return average

def shifts_generate(ranges):
    output=[]
    for i in range(ranges[0],ranges[1],1):
        for j in range(ranges[0],ranges[1],1):
            if i==0 or j==0:
                continue
            output.append([i,j])
    return output

(1) read image

In [3]:
image=util.imageRead("../Original_Images/lena.raw",16,16,1)
print(image)

[[[135]
  [125]
  [ 69]
  [ 83]
  [ 98]
  [ 99]
  [ 96]
  [ 95]
  [ 96]
  [ 89]
  [111]
  [132]
  [147]
  [134]
  [ 90]
  [ 64]]

 [[133]
  [119]
  [ 65]
  [ 80]
  [ 93]
  [ 96]
  [111]
  [132]
  [106]
  [ 81]
  [ 99]
  [128]
  [119]
  [159]
  [ 70]
  [ 17]]

 [[ 93]
  [115]
  [ 62]
  [ 77]
  [ 86]
  [ 91]
  [115]
  [169]
  [188]
  [111]
  [ 91]
  [ 98]
  [108]
  [ 93]
  [ 25]
  [ 52]]

 [[ 69]
  [116]
  [ 60]
  [ 80]
  [ 83]
  [ 92]
  [120]
  [170]
  [195]
  [193]
  [118]
  [ 76]
  [133]
  [ 43]
  [ 34]
  [125]]

 [[ 75]
  [122]
  [ 60]
  [ 89]
  [ 89]
  [ 95]
  [134]
  [159]
  [169]
  [178]
  [171]
  [176]
  [183]
  [ 37]
  [ 94]
  [140]]

 [[ 78]
  [126]
  [ 58]
  [ 96]
  [115]
  [ 99]
  [104]
  [ 95]
  [ 98]
  [157]
  [185]
  [167]
  [ 90]
  [ 50]
  [137]
  [135]]

 [[ 79]
  [126]
  [ 59]
  [ 82]
  [121]
  [ 79]
  [ 47]
  [ 40]
  [124]
  [176]
  [139]
  [ 80]
  [ 26]
  [ 95]
  [147]
  [136]]

 [[ 75]
  [127]
  [ 63]
  [ 82]
  [ 87]
  [ 40]
  [ 32]
  [112]
  [127]
  [160]
  [133]
  

(2) Subtract value from each pixel

In [4]:
imageSubtra=util.PixelSubtract(image,128)
#print(imageSubtra)

(3) apply two dimensions DCT on image blocks

In [5]:
imageBlocks=util.blocking(imageSubtra,8)
#print(imageBlocks)
imageDCT=[]
for i in imageBlocks:
    imageDCT.append(util.DCT(i))
print(np.array(imageDCT))

blocking: shape of data (16, 16, 1)
[[[ 1.55900000e+03  9.48463821e+01 -1.87336578e+02  6.83270645e+01
   -1.31500000e+02 -7.66684875e+01 -1.33469131e+02 -4.77050285e+01]
  [-7.30949097e+01 -6.97603378e+01 -1.22768951e+02 -1.12828624e+00
   -1.55936386e+02  3.20359383e+01 -7.09245758e+01  5.36691284e+00]
  [ 2.65392895e+01 -1.52541336e+02  2.76709423e+01 -1.29073990e+02
    4.62422967e-01 -6.29003677e+01 -2.87748642e+01  2.13681531e+00]
  [ 1.60901451e+01 -6.38765068e+01 -1.76308136e+01 -5.68040466e+01
    2.86537743e+01 -8.13562241e+01  3.21589088e+01 -3.85244255e+01]
  [-3.32500000e+01  2.16967583e+01  6.61935329e+00 -4.15022240e+01
    3.62500000e+01 -4.68822021e+01  4.40716362e+01 -3.24672928e+01]
  [ 7.27451859e+01 -4.96510887e+01  6.14581833e+01  8.56245136e+00
    7.32131004e+00  3.10255013e+01 -1.90533695e+01  2.76850452e+01]
  [ 8.23633957e+01 -4.67546844e+01  9.34751358e+01 -2.09240036e+01
    4.55395279e+01  1.40849056e+01  8.57905865e+00  6.50579977e+00]
  [-4.23704491e+01 

(4) Compute the quantized matrix for all four DCT coefficient blocks of the 16x16 Lena.raw image using 𝑄50

In [6]:
quantiDCT=[]
q50=util.quantiMatrix(50)
for i in imageDCT:
    print(np.around(i/q50).astype(int))


[[ 97   9 -19   4  -5  -2  -3  -1]
 [ -6  -6  -9   0  -6   1  -1   0]
 [  2 -12   2  -5   0  -1   0   0]
 [  1  -4  -1  -2   1  -1   0  -1]
 [ -2   1   0  -1   1   0   0   0]
 [  3  -1   1   0   0   0   0   0]
 [  2  -1   1   0   0   0   0   0]
 [ -1   1   0   0   0   0   1   0]]
[[ 65   3   6   4   3   1   1   1]
 [ 16   1   9 -11 -10  -1  -1   3]
 [  2  11  11   0   4  -1  -1   2]
 [-12   4   3   6  -2   1   0   1]
 [ -5  -5   4  -3   0   1   0   0]
 [  3   0  -1  -1   0   1   1  -2]
 [  1   0   0   0   0  -1   0   0]
 [  0   0  -1   1  -1   0  -1  -1]]
[[ 86 -17  -3   2   9   4   4   2]
 [  5   1  -1  -3  -2  -1  -1  -1]
 [  3   0   5  -1   0   0  -1   0]
 [ -1   0   0   0   0   0   0   0]
 [ -2  -1  -1   0   1   0   0   0]
 [ -2  -1  -1   0   1   0   1   0]
 [ -1  -1   0   0   0   0   1   0]
 [  0   0   0   0   0   0   0   0]]
[[ 75  23  -4   5   6  -4   0   0]
 [ -3  18 -15  -7  -4   0   0  -1]
 [ -4   3   5  -1  -1  -2  -3   2]
 [ -7   1  -2  -4   4   1   0   2]
 [  3   1   1   1

(5)Find quantization matrices 𝑄10 and 𝑄80 and show them in your report. Then compute the two corresponding quantized matrices for the “Lena” image and show 4 8x8 quantized coefficients blocks for each matrix.

In [7]:
q80=util.quantiMatrix(80)
q10=util.quantiMatrix(10)
print(q80)
print(q10)
print("q80")
for i in imageDCT:
    print(np.around(i/q80).astype(int))
print("q10")
for i in imageDCT:
    print(np.around(i/q10).astype(int))

[[ 6.4  4.4  4.   6.4  9.6 16.  20.4 24.4]
 [ 4.8  4.8  5.6  7.6 10.4 23.2 24.  22. ]
 [ 5.6  5.2  6.4  9.6 16.  22.8 27.6 22.4]
 [ 5.6  6.8  8.8 11.6 20.4 34.8 32.  24.8]
 [ 7.2  8.8 14.8 22.4 27.2 43.6 41.2 30.8]
 [ 9.6 14.  22.  25.6 32.4 41.6 45.2 36.8]
 [19.6 25.6 31.2 34.8 41.2 48.4 48.  40.4]
 [28.8 36.8 38.  39.2 44.8 40.  41.2 39.6]]
[[ 80.  55.  50.  80. 120. 200. 255. 305.]
 [ 60.  60.  70.  95. 130. 290. 300. 275.]
 [ 70.  65.  80. 120. 200. 285. 345. 280.]
 [ 70.  85. 110. 145. 255. 435. 400. 310.]
 [ 90. 110. 185. 280. 340. 545. 515. 385.]
 [120. 175. 275. 320. 405. 520. 565. 460.]
 [245. 320. 390. 435. 515. 605. 600. 505.]
 [360. 460. 475. 490. 560. 500. 515. 495.]]
q80
[[244  22 -47  11 -14  -5  -7  -2]
 [-15 -15 -22   0 -15   1  -3   0]
 [  5 -29   4 -13   0  -3  -1   0]
 [  3  -9  -2  -5   1  -2   1  -2]
 [ -5   2   0  -2   1  -1   1  -1]
 [  8  -4   3   0   0   1   0   1]
 [  4  -2   3  -1   1   0   0   0]
 [ -1   3   0   1   1  -1   1  -1]]
[[163   8  14   9   6   1

JPEG Compression Quality Factor

In [8]:
jpeglib.version.set('6b')
image=util.imageRead("../Original_Images/boat.bmp",512,512,1)
im = jpeglib.from_spatial(image)
name="boat100.jpeg"
qt=100
im.jpeg_color_space = jpeglib.JCS_GRAYSCALE
im.write_spatial(name,qt) #compress with quality N=100 in valid range [0, 100]
im_quanti = jpeglib.read_spatial(name)
util.PSNR(im.spatial,im_quanti.spatial,8)


58.49333016561671

Part C: Post-Processing of JPEG Encoded Images 

(a)Prepare for data

In [9]:
image1=util.imageRead("../Original_Images/camera_man.raw",256,256,1)
image2=util.imageRead("../Original_Images/pepper.raw",256,256,3)
image1_1=jpeglib.read_spatial("../JPEG_Images/camera_man1.jpg").spatial
image1_2=jpeglib.read_spatial("../JPEG_Images/camera_man2.jpg").spatial
image2_1=jpeglib.read_spatial("../JPEG_Images/pepper_1.jpg").spatial
image2_2=jpeglib.read_spatial("../JPEG_Images/pepper_2.jpg").spatial
print(util.PSNR(image1,image1_1,8))
print(util.PSNR(image1,image1_2,8))


32.73870329277156
34.383214456472494


In [10]:
height, width, channels=np.shape(image2)
print("image2_1")
average_PSNR=[]
for h in range(channels):
    average_PSNR.append(util.PSNR(image2[:,:,h],image2_1[:,:,h],24))
print(np.mean(average_PSNR))
print("image2_2")
average_PSNR=[]
for h in range(channels):
   average_PSNR.append(util.PSNR(image2[:,:,h],image2_2[:,:,h],24))
print(np.mean(average_PSNR))

#print(image2)

image2_1
127.31742544327544
image2_2
128.8235525466598


(b)Post-processing
(1)

In [27]:
print(util.PSNR(image1,util.boundary_filter(image1_1[:,:,0],8,1,2),8))
print(util.PSNR(image1,util.boundary_filter(image1_2[:,:,0],8,1,2),8))
cv2.imwrite('image1_1_boundary.png',util.boundary_filter(image1_1[:,:,0],8,1,2))
cv2.imwrite('image1_2_boundary.png',util.boundary_filter(image1_2[:,:,0],8,1,2))

27.99534996204911
28.018364192334637


True

In [12]:
height, width, channels=np.shape(image2)
print("image2_1")
average_PSNR=[]
for h in range(channels):
    average_PSNR.append(util.PSNR(image2[:,:,h],util.boundary_filter(image2_1[:,:,h],8,1,2),24))
print(np.mean(average_PSNR))
print("image2_2")
average_PSNR=[]
for h in range(channels):
   average_PSNR.append(util.PSNR(image2[:,:,h],util.boundary_filter(image2_2[:,:,h],8,1,2),24))
print(np.mean(average_PSNR))

image2_1
125.76596098998385
image2_2
126.26646693394935


(b)Post-processing
(2)

In [43]:
shift=shifts_generate([-3,4])
print("image1_1_oddSymm",util.PSNR(image1,reapply_jpeg(image1_1,shift,20,util.shift_image_odd_symmetric),8))
print("image1_1_replica",util.PSNR(image1,reapply_jpeg(image1_1,shift,20,util.shift_image_replication),8))
print("image1_1_Zero",util.PSNR(image1,reapply_jpeg(image1_1,shift,20,util.shift_image_zero_shift),8))
print("image1_2_oddSymm",util.PSNR(image1,reapply_jpeg(image1_2,shift,60,util.shift_image_odd_symmetric),8))
print("image1_2_replica",util.PSNR(image1,reapply_jpeg(image1_2,shift,60,util.shift_image_replication),8))
print("image1_2_Zero",util.PSNR(image1,reapply_jpeg(image1_2,shift,60,util.shift_image_zero_shift),8))

Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution:

image1_1_oddSymm 31.312675234592646
image1_1_replica 31.57048110109624
image1_1_Zero 31.2173881404367
image1_2_oddSymm 31.37394776447986
image1_2_replica 31.656189507602384
image1_2_Zero 31.317683052572967


In [42]:
print("image2_1_oddSymm",util.PSNR(image2,reapply_jpeg(image2_1,shift,20,util.shift_image_odd_symmetric),24))
print("image2_1_replica",util.PSNR(image2,reapply_jpeg(image2_1,shift,20,util.shift_image_replication),24))
print("image2_1_zero",util.PSNR(image2,reapply_jpeg(image2_1,shift,20,util.shift_image_zero_shift),24))
print("image2_2_oddSymm",util.PSNR(image2,reapply_jpeg(image2_2,shift,60,util.shift_image_odd_symmetric),24))
print("image2_2_replica",util.PSNR(image2,reapply_jpeg(image2_2,shift,60,util.shift_image_replication),24))
print("image2_2_zero",util.PSNR(image2,reapply_jpeg(image2_2,shift,60,util.shift_image_zero_shift),24))


Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution:

image2_1_oddSymm 124.10696000410276
image2_1_replica 124.10522057792691
image2_1_zero 124.1134459141992


Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG
Caution: quantization tables are too coarse for baseline JPEG


image2_2_oddSymm 124.0994370637518
image2_2_replica 124.10301149508307
image2_2_zero 124.1124873193236
